In [2]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [3]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 32
# Define directories for training and testing data
train_data_dir = "dataset/train"
test_data_dir = "dataset/test"

In [4]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [5]:
# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)



Found 1383 images belonging to 14 classes.
Found 1417 images belonging to 14 classes.


In [6]:
# # Define the ResNet50 base model
# def create_resnet_model():
#     resnet_model = ResNet50(
#         input_shape=(img_height, img_width, 3),  # Adjust input shape
#         include_top=False,  # Exclude the fully-connected layers
#         weights='imagenet'  # Pre-trained on ImageNet
#     )
#     resnet_model.trainable = False

#     model = Sequential([
#         resnet_model,
#         GlobalAveragePooling2D(),
#         Dense(64, activation='relu'),
#         BatchNormalization(),
#         Dropout(0.2),
#         Dense(14, activation='sigmoid')
#     ])
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     print(model.summary())
#     return model

# #MODEL fit
# model = create_resnet_model()
# model.fit(train_generator, epochs=10, validation_data=test_generator)
# #predict
# y_pred = model.predict(test_generator)
# y_pred = np.argmax(y_pred, axis=1)
# y_true = test_generator.classes
# #accuracy
# accuracy = accuracy_score(y_true, y_pred)
# print(f"Accuracy: {accuracy}")



In [7]:
# Define the VGG16 base model
def create_vgg_model():
    vgg_model = VGG16(
        input_shape=(img_height, img_width, 3),  # Adjust input shape
        include_top=False,  # Exclude the fully-connected layers
        weights='imagenet'  # Pre-trained on ImageNet
    )
    vgg_model.trainable = False

    model = Sequential([
        vgg_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(14, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


#MODEL fit
model = create_vgg_model()
model.fit(train_generator, epochs=10, validation_data=test_generator)
#predict
y_pred = model.predict(test_generator)
y_pred = np.argmax(y_pred, axis=1)
y_true = test_generator.classes
#accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

58889256/58889256 [==============================] - 11s 0us/step
Epoch 1/10
44/44 [==============================] - 36s 544ms/step - loss: 2.3050 - accuracy: 0.2458 - val_loss: 2.3794 - val_accuracy: 0.1842
Epoch 2/10
44/44 [==============================] - 20s 463ms/step - loss: 1.6747 - accuracy: 0.4519 - val_loss: 2.1746 - val_accuracy: 0.3049
Epoch 3/10
44/44 [==============================] - 21s 485ms/step - loss: 1.4381 - accuracy: 0.5192 - val_loss: 1.9357 - val_accuracy: 0.4375
Epoch 4/10
44/44 [==============================] - 20s 457ms/step - loss: 1.2730 - accuracy: 0.6001 - val_loss: 1.7483 - val_accuracy: 0.5208
Epoch 5/10
44/44 [==============================] - 28s 638ms/step - loss: 1.1626 - accuracy: 0.6291 - val_loss: 1.5184 - val_accuracy: 0.5787
Epoch 6/10
44/44 [==============================] - 26s 580ms/step - loss: 1.0736 - accuracy: 0.6558 - val_loss: 1.3715 - val_accuracy: 0.6394
Epoch 7/10
44/44 [==============================] - 23s 523ms/step - loss: 0

In [8]:
# Wrap Keras models as scikit-learn compatible classifiers
# resnet_classifier = KerasClassifier(build_fn=create_resnet_model, epochs=8, batch_size=batch_size, verbose=0)
# vgg_classifier = KerasClassifier(build_fn=create_vgg_model, epochs=8, batch_size=batch_size, verbose=0)


In [9]:
# Combine the models into a voting ensemble
# ensemble = VotingClassifier(estimators=[('resnet', resnet_classifier), ('vgg', vgg_classifier)], voting='hard')


In [10]:

# # Train the ensemble
# ensemble.fit(train_features, train_labels)


In [11]:
# # Evaluate the ensemble
# predictions = ensemble.predict(test_generator)
# true_labels = test_generator.classes
# accuracy = accuracy_score(true_labels, predictions)
# print("Ensemble Accuracy:", accuracy)
